# - 𝐆𝐞𝐧𝐞𝐫𝐚𝐭𝐢𝐯𝐞 𝐀𝐝𝐯𝐞𝐫𝐬𝐚𝐫𝐢𝐚𝐥 𝐍𝐞𝐭𝐰𝐨𝐫𝐤𝐬..

In [1]:
# import library :
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input
from tensorflow.keras.layers import LeakyReLU, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

In [2]:
# Load the mnist datasets :
(x_train, _), (_, _) = mnist.load_data()

In [3]:
# Normlize the data :
x_train = (x_train.astype(np.float32) - 127.5) / 127.5
x_train = x_train.reshape(x_train.shape[0], 784)

In [4]:
# Define the generator model :
generator = Sequential([
    Dense(256, input_dim=100),
    LeakyReLU(alpha=0.2),
    BatchNormalization(),
    Dense(512),
    LeakyReLU(alpha=0.2),
    BatchNormalization(),
    Dense(1024),
    LeakyReLU(alpha=0.2),
    BatchNormalization(),
    Dense(784, activation="tanh"),
    Reshape((28, 28))
])

In [5]:
# Define the discriminator model:
descriminator = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(1024),
    LeakyReLU(alpha=0.2),
    Dense(512),
    LeakyReLU(alpha=0.2),
    Dense(256),
    LeakyReLU(alpha=0.2),
    Dense(1, activation="sigmoid")
])

In [6]:
# compile the discriminator :
descriminator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
                      loss="binary_crossentropy", metrics=["Accuracy"])

In [7]:
# compile the GAN model :
descriminator.trainable = False
gan_input = Input(shape=(100, ))
x = generator(gan_input)
gan_output = descriminator(x)
gan = Model(gan_input, gan_output)
gan.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
           loss="binary_crossentropy")

In [12]:
# Function to train the GAN :
def train_gan(epochs=1, batch_size=128):
    # Calculate the number of batches per epochs.
    batch_count = x_train.shape[0] // batch_size
    
    for e in range(epochs):
        for _ in range(batch_count):
            # Generate random noise as input for the generator.
            noise = np.random.normal(0, 1, size=[batch_size, 100])
            
            # Generate fake images from generator.
            generated_images = generator.predict(noise)
            
            # get random batch of real images from the dadasets.
            batch_index = np.random.randint(0, x_train.shape[0], batch_size)
            real_images = x_train[batch_index]
            
            # concatenate real and fake images.
            x = np.concatenate([real_images, generated_images])
            
            # Labels for generated and real data.
            y_dis = np.zeros(2 * batch_size)
            y_dis[:batch_size] = 0.9     # one-sided label encoding.
            
            #Train the descriminator.
            descriminator.trainable = True
            d_los = descriminator.train_on_batch(x, y_dis)
            
            #Train the descriminator.in the generator (via the GAN model)
            noise = np.random.normal(0, 1, size=[batch_size, 100])
            y_gen = np.ones(batch_size)
            descriminator.trainable = False
            g_loss = gan.train_on_batch(noise, y_gen)
            
            # Print the progress and save the generated iamges.
            print(f"Epoch {e+1}, Discriminatoe Loss: {d_los[0]}, Generator Loss: {g_loss}")
            if e % 10 == 0:
                plot_generated_images(e, generator)

In [14]:
# Function to plot generated images.
def plot_generated_images(epoch, generator, example=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, size=[example, 100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(example, 28, 28)
    
    plt.figure(figsize=figsize)
    for i in range(example):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation="nearest", cmap="gray")
        plt.axis("off")
    plt.tight_layout()
    plt.savefig(f"gan_generated_image_epoch_{epoch}.png")
    plt.show()
    
# Train the GAN model:
train_gan(epochs=100, batch_size=128)

4/4 [==============================] - 0s 5ms/step


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)

In [11]:
# Train the GAN model:
train_gan(epochs=100, batch_size=128)

4/4 [==============================] - 0s 6ms/step


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)

In [15]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input
from tensorflow.keras.layers import LeakyReLU, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

# Load the MNIST dataset
(X_train, _), (_, _) = mnist.load_data()

# Normalize the data
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = X_train.reshape(X_train.shape[0], 784)

# Define the generator model
generator = Sequential([
    Dense(256, input_dim=100),
    LeakyReLU(alpha=0.2),
    BatchNormalization(),
    Dense(512),
    LeakyReLU(alpha=0.2),
    BatchNormalization(),
    Dense(1024),
    LeakyReLU(alpha=0.2),
    BatchNormalization(),
    Dense(784, activation='tanh'),
    Reshape((28, 28))
])

# Define the discriminator model
discriminator = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(1024),
    LeakyReLU(alpha=0.2),
    Dense(512),
    LeakyReLU(alpha=0.2),
    Dense(256),
    LeakyReLU(alpha=0.2),
    Dense(1, activation='sigmoid')
])

# Compile the discriminator
discriminator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
                      loss='binary_crossentropy', metrics=['accuracy'])

# Compile the GAN model
discriminator.trainable = False
gan_input = Input(shape=(100,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(gan_input, gan_output)
gan.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
            loss='binary_crossentropy')

# Function to train the GAN
def train_gan(epochs=1, batch_size=128):
    # Calculate the number of batches per epoch
    batch_count = X_train.shape[0] // batch_size
    
    for e in range(epochs):
        for _ in range(batch_count):
            # Generate random noise as input for the generator
            noise = np.random.normal(0, 1, size=[batch_size, 100])
            
            # Generate fake images using the generator
            generated_images = generator.predict(noise)
            
            # Get a random batch of real images from the dataset
            batch_idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_images = X_train[batch_idx]
            
            # Concatenate real and fake images
            X = np.concatenate([real_images, generated_images])
            
            # Labels for generated and real data
            y_dis = np.zeros(2 * batch_size)
            y_dis[:batch_size] = 0.9  # One-sided label smoothing
            
            # Train the discriminator
            discriminator.trainable = True
            d_loss = discriminator.train_on_batch(X, y_dis)
            
            # Train the generator (via the GAN model)
            noise = np.random.normal(0, 1, size=[batch_size, 100])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, y_gen)
            
        # Print the progress and save the generated images
        print(f"Epoch {e+1}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}")
        if e % 10 == 0:
            plot_generated_images(e, generator)

# Function to plot generated images
def plot_generated_images(epoch, generator, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, size=[examples, 100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f'gan_generated_image_epoch_{epoch}.png')
    plt.show()

# Train the GAN
train_gan(epochs=100, batch_size=128)

4/4 [==============================] - 0s 5ms/step


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)